In [1]:
import geopandas as gpd
import datetime

In [2]:
import sys
sys.path.append('..')

In [3]:
import mysecrets
import fetch_from_cluster

/Users/nikhilsrajan/NASA-Harvest/project/fetch_satdata/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
eth_s2grids_gdf = gpd.read_file('../../ethiopia/data/ethiopia_s2_grids_esa_stats.geojson')

In [5]:
S2GRIDS_OF_INTEREST = [
    '17b4c3c', # 4 tiles
    '17b37fc', # 2 tiles ?
    '164c59c', # corner
]

In [6]:
eth_s2grids_gdf.set_index('id').loc[
    S2GRIDS_OF_INTEREST, 'geometry'
].reset_index().to_file('../data/selected_eth_grids.geojson')

In [7]:
selected_s2grid_gdf = eth_s2grids_gdf[eth_s2grids_gdf['id'] == S2GRIDS_OF_INTEREST[0]][['id', 'geometry']]

In [8]:
catalog_gdf = fetch_from_cluster.query_sentinel2_catalog(
    shapes_gdf = selected_s2grid_gdf,
    # shapes_gdf = eth_s2grids_gdf.set_index('id').loc[
    #     S2GRIDS_OF_INTEREST, 'geometry'
    # ].reset_index(),
    startdate = datetime.datetime(2021, 3, 1),
    enddate = datetime.datetime(2021, 3, 5),
    sshcreds = mysecrets.SSH_UMD_SASIRAJANN,
    satellite_folderpath = '../data/cluster_files/satellite',
    overwrite_catalog = False,
)

In [9]:
local_download_filepaths_df \
= fetch_from_cluster.download_intersecting_sentinel2_tiles_from_cluster(
    shapes_gdf = selected_s2grid_gdf,
    startdate = datetime.datetime(2021, 3, 1),
    enddate = datetime.datetime(2021, 3, 5),
    sshcreds = mysecrets.SSH_UMD_SASIRAJANN,
    satellite_folderpath = '../data/cluster_files/satellite',
    bands = ['B02', 'B03', 'B04', 'B08'],
    overwrite_catalog = False,
    njobs = 4,
)

100%|██████████| 16/16 [10:28<00:00, 39.31s/it]
